# Pinterest ETL pipeline (spark with redis)

### Import packages

In [1]:
import findspark
findspark.init()
import pyspark
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
from cassandra.cluster import Cluster
import multiprocessing
from pyspark.sql import SparkSession
from pyspark import SparkConf
from pyspark.sql.functions import when, regexp_replace, regexp_extract, col
import pyspark.sql.functions as F
import requests
from time import sleep
import random
from multiprocessing import Process
import boto3
import json
import sqlalchemy
import uuid
import os 
import timeit

### Loading data from source

In [ ]:
class AWSDBConnector:

    def __init__(self):

        self.HOST = "pinterestdbreadonly.cq2e8zno855e.eu-west-1.rds.amazonaws.com"
        self.USER = 'project_user'
        self.PASSWORD = <password>
        self.DATABASE = 'pinterest_data'
        self.PORT = 3306
        
    def create_db_connector(self):
        engine = sqlalchemy.create_engine(f"mysql+pymysql://{self.USER}:{self.PASSWORD}@{self.HOST}:{self.PORT}/{self.DATABASE}?charset=utf8mb4")
        return engine


new_connector = AWSDBConnector()
engine = new_connector.create_db_connector()

In [ ]:
# create uuid for unique file names
id = uuid.uuid4()
str(id)

In [ ]:
# save json file to designated location
n=0
while True:

    
    selected_row = engine.execute(f"SELECT * FROM pinterest_data LIMIT {n}, 1")
    for row in selected_row:
        result = dict(row)
    n+=1
    file_name = str(id) + "_" + str(n)+ "." + "json"
    path_to_the_file = '/Users/tanyali/Documents/practice/data/' + file_name
    file = json.dumps(result)
    with open(path_to_the_file, 'w') as outfile:
        outfile.write(file)
    # print(result)

### Initialise pyspark session and loading data and save to redis table

In [2]:
# initialise pyspark session
starttime = timeit.default_timer()
print("The start time is :",starttime)
spark = SparkSession.builder \
        .master(f"local[{multiprocessing.cpu_count()}]") \
        .appName("pin_app") \
        .getOrCreate()
        # .master("local[*]")\
        
        

# hadoopConf = sc._jsc.hadoopConfiguration()
# hadoopConf.set("fs.s3.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
# hadoopConf.set('fs.s3a.access.key',<your access key>)
# hadoopConf.set('fs.s3a.secret.key', <your secret key>)
# hadoopConf.set('spark.hadoop.fs.s3a.aws.credentials.provider', 'org.apache.hadoop.fs.s3a.SimpleAWSCredentialsProvider')
print("The time difference is :", timeit.default_timer() - starttime)




The start time is : 17425.14962893


23/01/11 18:29:14 WARN Utils: Your hostname, h1m1w1-CRAWLER-E10 resolves to a loopback address: 127.0.1.1; using 192.168.1.121 instead (on interface wlp0s20f3)
23/01/11 18:29:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/01/11 18:29:15 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


The time difference is : 7.097728323999036


In [3]:
# load data as json files
starttime = timeit.default_timer()
print("The start time is :",starttime)
df = spark.read.json("s3a://basicaccountstack-pinterestdataeng-proje-datalake-tcvpj2nf0cpq/*.json")
df.show(2)
print("The time difference is :", timeit.default_timer() - starttime)


The start time is : 17443.755173812


+----------+--------------------+----------+--------------+--------------------+-----+-----------------+-------------+--------------------+--------------------+--------------------+--------------------+
|  category|         description|downloaded|follower_count|           image_src|index|is_image_or_video|  poster_name|       save_location|            tag_list|               title|           unique_id|
+----------+--------------------+----------+--------------+--------------------+-----+-----------------+-------------+--------------------+--------------------+--------------------+--------------------+
|   tattoos|Elmira Kruger sha...|         1|           848|https://i.pinimg....| 8822|            image|Alicia Keller|Local save in /da...|Arm Sleeve Tattoo...|Elmira Kruger on ...|db80a4e4-293a-45c...|
|home-decor|Вместе с Русланом...|         1|           35k|https://i.pinimg....| 5996|            image|  INMYROOM.RU|Local save in /da...|Scandinavian Inte...|Где найти и как о...|4081ef0

In [5]:
# save dataframe to redis table
starttime = timeit.default_timer()
print("The start time is :",starttime)
df.write.format("org.apache.spark.sql.redis").option("table", "pins").option("key.column", "index").save()
print("The time difference is :", timeit.default_timer() - starttime)


The start time is : 17520.828128229


The time difference is : 9.510933825000393


In [6]:
# read the dataframe frm redis table 
starttime = timeit.default_timer()
print("The start time is :",starttime)
data = spark.read.format("org.apache.spark.sql.redis").option("table", "pins").option("key.column", "index").load()
print("The time difference is :", timeit.default_timer() - starttime)


The start time is : 17543.232067959
The time difference is : 0.013840228999470128


### Data cleaning / Remove duplicates 

In [7]:
# starting ETL
starttime = timeit.default_timer()
print("The start time is :",starttime)
data =data.dropDuplicates()
data.na.drop(subset=["index"]) \
   .show(truncate=False)
print("The time difference is :", timeit.default_timer() - starttime)


The start time is : 17550.987833464


+--------------+-----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------+--------------+----------------------------------------------------------------------------+-----+------------------------------+----------------------------------------------------------------------------------------+----------------------------------+--------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+------------------------------------------------------------------------------+------------------------------------+
|category      |description                                                                                                                                                                        |downloaded|follower_count|image_src         

In [8]:
starttime = timeit.default_timer()
print("The start time is :",starttime)
for i in data.columns:
    data.groupBy(i).count().orderBy("count", ascending=False).show()
print("The time difference is :", timeit.default_timer() - starttime)


The start time is : 17564.005831874


+--------------+-----+
|      category|count|
+--------------+-----+
|     christmas|  999|
|       tattoos|  970|
|        quotes|  910|
|           art|  892|
|     education|  863|
|    home-decor|  861|
|        travel|  859|
|  mens-fashion|  852|
|      vehicles|  838|
|diy-and-crafts|  830|
|       finance|  786|
|event-planning|  748|
|        beauty|  746|
+--------------+-----+

+--------------------+-----+
|         description|count|
+--------------------+-----+
|No description av...|  895|
|No description av...|  182|
|Relationship Rule...|   40|
|Our mission at K&...|   24|
|Is your order tim...|   14|
|Imagine your birt...|   14|
|Top Apps for Even...|   14|
|Use our easy step...|   12|
|It doesn't matter...|   12|
|Provide a creativ...|   12|
|Macho Moda: Blog ...|    9|
|Afrocentric Bathr...|    9|
|Getting a tattoo ...|    7|
|Tara Jane Crandon...|    7|
|Mental health tat...|    7|
|Healing takes str...|    7|
|There are more wa...|    7|
|Made of quality w...|    7|

+-----+-----+
|index|count|
+-----+-----+
| 1677|    1|
| 8075|    1|
| 1950|    1|
| 9715|    1|
| 4590|    1|
| 9444|    1|
| 7544|    1|
| 6424|    1|
| 3937|    1|
| 2989|    1|
| 2909|    1|
| 7679|    1|
| 6489|    1|
| 3845|    1|
| 6860|    1|
| 8636|    1|
| 6856|    1|
|10430|    1|
| 7664|    1|
| 4551|    1|
+-----+-----+
only showing top 20 rows

+--------------------+-----+
|   is_image_or_video|count|
+--------------------+-----+
|               image| 9920|
|multi-video(story...|  895|
|               video|  332|
|                null|    7|
+--------------------+-----+

+--------------------+-----+
|         poster_name|count|
+--------------------+-----+
|           YourTango|  244|
|         Bored Panda|  140|
|    Our Mindful Life|   88|
|     User Info Error|   88|
|             Walmart|   86|
|   The Minds Journal|   86|
|            BuzzFeed|   81|
|Dividends Diversi...|   68|
|         Boop Decals|   60|
|         Next Luxury|   54|
|           SOCIETY19|   53|

In [9]:
# total rows
starttime = timeit.default_timer()
print("The start time is :",starttime)
print(data.count())
print("The time difference is :", timeit.default_timer() - starttime)

The start time is : 17580.108028812
11154
The time difference is : 0.37152256600165856


In [10]:
# remove duplicate rows
starttime = timeit.default_timer()
print("The start time is :",starttime)
data =data.dropDuplicates()
print("The time difference is :", timeit.default_timer() - starttime)


The start time is : 17583.941454852
The time difference is : 0.003681780999613693


In [11]:
# total rows after removing duplicates.
starttime = timeit.default_timer()
print("The start time is :",starttime)
print(df.count())
print("The time difference is :", timeit.default_timer() - starttime)


The start time is : 17587.375368852


20226
The time difference is : 7.383094689997961


### Data transformation

In [12]:
# dataset schema
starttime = timeit.default_timer()
print("The start time is :",starttime)
data.printSchema()
print("The time difference is :", timeit.default_timer() - starttime)


The start time is : 17604.565206599
root
 |-- category: string (nullable = true)
 |-- description: string (nullable = true)
 |-- downloaded: long (nullable = true)
 |-- follower_count: string (nullable = true)
 |-- image_src: string (nullable = true)
 |-- index: long (nullable = true)
 |-- is_image_or_video: string (nullable = true)
 |-- poster_name: string (nullable = true)
 |-- save_location: string (nullable = true)
 |-- tag_list: string (nullable = true)
 |-- title: string (nullable = true)
 |-- unique_id: string (nullable = true)

The time difference is : 0.002019122999627143


In [14]:
starttime = timeit.default_timer()
print("The start time is :",starttime)
print(data.dtypes)
print("The time difference is :", timeit.default_timer() - starttime)


The start time is : 17610.936296723
[('category', 'string'), ('description', 'string'), ('downloaded', 'bigint'), ('follower_count', 'string'), ('image_src', 'string'), ('index', 'bigint'), ('is_image_or_video', 'string'), ('poster_name', 'string'), ('save_location', 'string'), ('tag_list', 'string'), ('title', 'string'), ('unique_id', 'string')]
The time difference is : 0.0005221339997660834


In [15]:
# column names
starttime = timeit.default_timer()
print("The start time is :",starttime)
print(df.columns)
print("The time difference is :", timeit.default_timer() - starttime)


The start time is : 17611.066702423
['category', 'description', 'downloaded', 'follower_count', 'image_src', 'index', 'is_image_or_video', 'poster_name', 'save_location', 'tag_list', 'title', 'unique_id']
The time difference is : 0.0037685720017179847


In [16]:
# check errors in each column
starttime = timeit.default_timer()
print("The start time is :",starttime)
for i in df.columns:
    data.groupBy(i).count().orderBy("count", ascending=False).show()
print("The time difference is :", timeit.default_timer() - starttime)


The start time is : 17611.190124636
+--------------+-----+
|      category|count|
+--------------+-----+
|     christmas|  999|
|       tattoos|  970|
|        quotes|  910|
|           art|  892|
|     education|  863|
|    home-decor|  861|
|        travel|  859|
|  mens-fashion|  852|
|      vehicles|  838|
|diy-and-crafts|  830|
|       finance|  786|
|event-planning|  748|
|        beauty|  746|
+--------------+-----+

+--------------------+-----+
|         description|count|
+--------------------+-----+
|No description av...|  895|
|No description av...|  182|
|Relationship Rule...|   40|
|Our mission at K&...|   24|
|Is your order tim...|   14|
|Imagine your birt...|   14|
|Top Apps for Even...|   14|
|Use our easy step...|   12|
|It doesn't matter...|   12|
|Provide a creativ...|   12|
|Macho Moda: Blog ...|    9|
|Afrocentric Bathr...|    9|
|Getting a tattoo ...|    7|
|Tara Jane Crandon...|    7|
|Mental health tat...|    7|
|Healing takes str...|    7|
|There are more wa...

In [17]:
# following columns with errors.
starttime = timeit.default_timer()
print("The start time is :",starttime)
for i in [
 'description',
 'is_image_or_video',
 'poster_name',
 'save_location',
 'tag_list',
 'title',
 ]:
    data.groupBy(i).count().orderBy("count", ascending=False).show()
print("The time difference is :", timeit.default_timer() - starttime)


The start time is : 17618.049603949
+--------------------+-----+
|         description|count|
+--------------------+-----+
|No description av...|  895|
|No description av...|  182|
|Relationship Rule...|   40|
|Our mission at K&...|   24|
|Is your order tim...|   14|
|Imagine your birt...|   14|
|Top Apps for Even...|   14|
|Use our easy step...|   12|
|It doesn't matter...|   12|
|Provide a creativ...|   12|
|Macho Moda: Blog ...|    9|
|Afrocentric Bathr...|    9|
|Getting a tattoo ...|    7|
|Tara Jane Crandon...|    7|
|Mental health tat...|    7|
|Healing takes str...|    7|
|There are more wa...|    7|
|Made of quality w...|    7|
|Sunsum® Manifesta...|    6|
|Dragon tattoos ar...|    6|
+--------------------+-----+
only showing top 20 rows

+--------------------+-----+
|   is_image_or_video|count|
+--------------------+-----+
|               image| 9920|
|multi-video(story...|  895|
|               video|  332|
|                null|    7|
+--------------------+-----+

+--------

In [18]:
# replace error value or empty value with None
starttime = timeit.default_timer()
print("The start time is :",starttime)
data = data.replace({'No description available Story format': None}, subset = ['description'])\
       .replace({'No description available': None}, subset = ['description'])

data.groupBy('description').count().orderBy("count", ascending=False).show(3)
print("The time difference is :", timeit.default_timer() - starttime)


The start time is : 17620.927145757
+--------------------+-----+
|         description|count|
+--------------------+-----+
|                null| 1077|
|Relationship Rule...|   40|
|Our mission at K&...|   24|
+--------------------+-----+
only showing top 3 rows

The time difference is : 0.6642079880002711


In [19]:
# display error values in 'poster_name' column
starttime = timeit.default_timer()
print("The start time is :",starttime)
data.groupBy('poster_name').count().orderBy("count", ascending=False).show(5)
print("The time difference is :", timeit.default_timer() - starttime)


The start time is : 17621.666888797
+----------------+-----+
|     poster_name|count|
+----------------+-----+
|       YourTango|  244|
|     Bored Panda|  140|
|Our Mindful Life|   88|
| User Info Error|   88|
|         Walmart|   86|
+----------------+-----+
only showing top 5 rows

The time difference is : 0.6700546739994024


In [20]:
# replace error values "User Info Error" in column 'poster_name' with null
starttime = timeit.default_timer()
print("The start time is :",starttime)
data = data.replace({'User Info Error': None}, subset = ['poster_name'])
data.groupBy('poster_name').count().orderBy("count", ascending=False).show(4)
print("The time difference is :", timeit.default_timer() - starttime)


The start time is : 17622.405864725
+----------------+-----+
|     poster_name|count|
+----------------+-----+
|       YourTango|  244|
|     Bored Panda|  140|
|            null|   88|
|Our Mindful Life|   88|
+----------------+-----+
only showing top 4 rows

The time difference is : 0.6347206450009253


In [21]:
# display error values in 'tag_list' column
starttime = timeit.default_timer()
print("The start time is :",starttime)
data.groupBy('tag_list').count().orderBy("count", ascending=False).show(5)
print("The time difference is :", timeit.default_timer() - starttime)


The start time is : 17623.104328176
+--------------------+-----+
|            tag_list|count|
+--------------------+-----+
|N,o, ,T,a,g,s, ,A...|  171|
|Stop Motion,Hobbi...|    6|
|Weekly Budget,Bud...|    3|
|Love My Husband Q...|    2|
|Auto Design,Autom...|    2|
+--------------------+-----+
only showing top 5 rows

The time difference is : 0.5105266909995407


In [22]:
# get error value in tag_list
starttime = timeit.default_timer()
print("The start time is :",starttime)
tag = data.groupBy('tag_list').count().orderBy("count", ascending=False)
print(tag.collect()[0][0])
print("The time difference is :", timeit.default_timer() - starttime)


The start time is : 17623.68373122
N,o, ,T,a,g,s, ,A,v,a,i,l,a,b,l,e
The time difference is : 1.4744064119986433


In [23]:
# replace error values with null
starttime = timeit.default_timer()
print("The start time is :",starttime)
data = data.replace({'N,o, ,T,a,g,s, ,A,v,a,i,l,a,b,l,e': None}, subset = ['tag_list'])
# display tag list values after remove error values
data.groupBy('tag_list').count().orderBy("count", ascending=False).show(4)
print("The time difference is :", timeit.default_timer() - starttime)


The start time is : 17625.244055153
+--------------------+-----+
|            tag_list|count|
+--------------------+-----+
|                null|  171|
|Stop Motion,Hobbi...|    6|
|Weekly Budget,Bud...|    3|
|Love My Husband Q...|    2|
+--------------------+-----+
only showing top 4 rows

The time difference is : 0.7939933039997413


In [24]:
# display error values in 'title' column
starttime = timeit.default_timer()
print("The start time is :",starttime)
data.groupBy('title').count().orderBy("count", ascending=False).show(5)
print("The time difference is :", timeit.default_timer() - starttime)


The start time is : 17626.120727629
+--------------------+-----+
|               title|count|
+--------------------+-----+
|No Title Data Ava...|   88|
|  Relationship Rules|   17|
|Top Apps for Even...|   14|
|DIY Abstract Hear...|   12|
|18 Daily Motivati...|   10|
+--------------------+-----+
only showing top 5 rows

The time difference is : 0.5423302529998182


In [25]:
# get error value in title column
starttime = timeit.default_timer()
print("The start time is :",starttime)
tag = data.groupBy('title').count().orderBy("count", ascending=False)
tag.collect()[0][0]
print("The time difference is :", timeit.default_timer() - starttime)


The start time is : 17626.717487712
The time difference is : 0.8358624979991873


In [26]:
# replace error value with null
starttime = timeit.default_timer()
print("The start time is :",starttime)
data = data.replace({'No Title Data Available': None}, subset = ['title'])
data.groupBy('title').count().orderBy("count", ascending=False).show(4)
print("The time difference is :", timeit.default_timer() - starttime)


The start time is : 17627.879071349
+--------------------+-----+
|               title|count|
+--------------------+-----+
|                null|   88|
|  Relationship Rules|   17|
|Top Apps for Even...|   14|
|DIY Abstract Hear...|   12|
+--------------------+-----+
only showing top 4 rows

The time difference is : 0.9163785349992395


In [27]:
# display error value in save_location column
starttime = timeit.default_timer()
print("The start time is :",starttime)
data.groupBy('save_location').count().orderBy("count", ascending=False).show(50)
print("The time difference is :", timeit.default_timer() - starttime)


The start time is : 17628.90459745
+--------------------+-----+
|       save_location|count|
+--------------------+-----+
|Local save in /da...|  999|
|Local save in /da...|  970|
|Local save in /da...|  910|
|Local save in /da...|  892|
|Local save in /da...|  863|
|Local save in /da...|  861|
|Local save in /da...|  859|
|Local save in /da...|  852|
|Local save in /da...|  838|
|Local save in /da...|  830|
|Local save in /da...|  786|
|Local save in /da...|  748|
|Local save in /da...|  746|
+--------------------+-----+

The time difference is : 0.6198810569985653


In [28]:
# get example of error value in save_location column
starttime = timeit.default_timer()
print("The start time is :",starttime)
location_column = data.groupBy('save_location').count().orderBy("count", ascending=False)
print(location_column.collect()[0][0])
print("The time difference is :", timeit.default_timer() - starttime)


The start time is : 17629.598207864
Local save in /data/christmas
The time difference is : 0.3660508569992089


In [29]:
# replace/remove error part of the values 
starttime = timeit.default_timer()
print("The start time is :",starttime)
data = data.withColumn('save_location', regexp_replace('save_location', 'Local save in ', '')) 
data.groupBy('save_location').count().orderBy("count", ascending=False).show()
print("The time difference is :", timeit.default_timer() - starttime)


The start time is : 17630.023339833
+--------------------+-----+
|       save_location|count|
+--------------------+-----+
|     /data/christmas|  999|
|       /data/tattoos|  970|
|        /data/quotes|  910|
|           /data/art|  892|
|     /data/education|  863|
|    /data/home-decor|  861|
|        /data/travel|  859|
|  /data/mens-fashion|  852|
|      /data/vehicles|  838|
|/data/diy-and-crafts|  830|
|       /data/finance|  786|
|/data/event-planning|  748|
|        /data/beauty|  746|
+--------------------+-----+

The time difference is : 0.5866127240005881


In [30]:
# display error and null value
starttime = timeit.default_timer()
print("The start time is :",starttime)
data.groupBy('image_src').count().orderBy("count", ascending=False).show(4)
print("The time difference is :", timeit.default_timer() - starttime)


The start time is : 17630.674258521
+--------------------+-----+
|           image_src|count|
+--------------------+-----+
|    Image src error.|  323|
|                null|    7|
|https://i.pinimg....|    2|
|https://i.pinimg....|    2|
+--------------------+-----+
only showing top 4 rows

The time difference is : 0.5349585589974595


In [31]:
# replace error values with null
starttime = timeit.default_timer()
print("The start time is :",starttime)
data = data.replace({'Image src error.': None}, subset = ['image_src'])
# display image_src values
data.groupBy('image_src').count().orderBy("count", ascending=False).show(4)
print("The time difference is :", timeit.default_timer() - starttime)



The start time is : 17631.271712131
+--------------------+-----+
|           image_src|count|
+--------------------+-----+
|                null|  330|
|https://i.pinimg....|    2|
|https://i.pinimg....|    2|
|https://i.pinimg....|    2|
+--------------------+-----+
only showing top 4 rows

The time difference is : 0.6417604529997334


In [32]:
# drop the rows with null values in 'image_src' column, as we don't want a row without a pin
starttime = timeit.default_timer()
print("The start time is :",starttime)
data = data.na.drop(subset=["image_src"])
# display image_src values after removing nul values
data.groupBy('image_src').count().orderBy("count", ascending=False).show(5)
print("The time difference is :", timeit.default_timer() - starttime)



The start time is : 17631.978432928
+--------------------+-----+
|           image_src|count|
+--------------------+-----+
|https://i.pinimg....|    2|
|https://i.pinimg....|    2|
|https://i.pinimg....|    2|
|https://i.pinimg....|    2|
|https://i.pinimg....|    2|
+--------------------+-----+
only showing top 5 rows

The time difference is : 0.5652917849984078


In [33]:
# check errors in column
starttime = timeit.default_timer()
print("The start time is :",starttime)
data.groupBy('follower_count').count().orderBy("count", ascending=False).show()
print("The time difference is :", timeit.default_timer() - starttime)


The start time is : 17632.588329142
+--------------+-----+
|follower_count|count|
+--------------+-----+
|            2M|  374|
|            2k|  325|
|            4k|  250|
|          942k|  244|
|            5k|  232|
|            7k|  227|
|            3k|  219|
|            1k|  219|
|            5M|  215|
|            1M|  178|
|            6k|  172|
|            9k|  141|
|            8k|  126|
|           28k|  115|
|           10k|  113|
|           20k|   99|
|           19k|   93|
|          190k|   90|
|           13k|   90|
|           11k|   88|
+--------------+-----+
only showing top 20 rows

The time difference is : 0.4390092399989953


In [34]:
# create a pandas dataframe which run much faster
starttime = timeit.default_timer()
print("The start time is :",starttime)
pddf = data.toPandas()
print("The time difference is :", timeit.default_timer() - starttime)


The start time is : 17633.092080884


The time difference is : 2.0102925960018183


In [35]:
# check if there is any characters in the follower_count column
starttime = timeit.default_timer()
print("The start time is :",starttime)
a = []
for i in pddf['follower_count'].to_list():
    if i is not None:
        for char in i:
            if  not char.isdigit():
                a.append(char)
# print all the characters in this columns
print(set(a))
print("The time difference is :", timeit.default_timer() - starttime)


The start time is : 17635.148992818
{'k', 'o', 'E', 's', 'n', 'U', 'e', ' ', 'f', 'M', 'I', 'r'}
The time difference is : 0.00598904899743502


In [36]:
# check each character 
starttime = timeit.default_timer()
print("The start time is :",starttime)
for i in pddf['follower_count'].to_list():
    if 'E' in i:
        # print the item with error
        print(i)
print("The time difference is :", timeit.default_timer() - starttime)


The start time is : 17635.228712991
User Info Error
The time difference is : 0.0030868169997120276


In [37]:
# replace 'User Info Error' with 0
starttime = timeit.default_timer()
print("The start time is :",starttime)
data = data.replace({'User Info Error': "0"}, subset = ['follower_count'])
print("The time difference is :", timeit.default_timer() - starttime)


The start time is : 17635.30819853
The time difference is : 0.015762486997118685


In [38]:
# check it again if there is any characters in the follower_count column
starttime = timeit.default_timer()
print("The start time is :",starttime)
pddf = data.toPandas()
a = []
for i in pddf['follower_count'].to_list():
    if i is not None:
        for char in i:
            if  not char.isdigit():
                a.append(char)
print("The time difference is :", timeit.default_timer() - starttime)


The start time is : 17635.401188226
The time difference is : 0.9389763160033908


In [39]:
# only 'M' 'k' need to be replaced by '000000' and '000' accordingly
starttime = timeit.default_timer()
print("The start time is :",starttime)
print("number of 'M' or 'k' is: ", len(a))
print(set(a))
print("The time difference is :", timeit.default_timer() - starttime)


The start time is : 17636.400503512
number of 'M' or 'k' is:  8986
{'M', 'k'}
The time difference is : 0.0007851370028220117


In [40]:
# replaced 'M', 'k' by '000000' and '000'
starttime = timeit.default_timer()
print("The start time is :",starttime)
data = data.withColumn('follower_count', 
    when(data.follower_count.endswith('M'),regexp_replace(data.follower_count,'M','000000')) \
   .when(data.follower_count.endswith('k'),regexp_replace(data.follower_count,'k','000')) \
   .otherwise(data.follower_count)) 
print("The time difference is :", timeit.default_timer() - starttime)



The start time is : 17636.519039217
The time difference is : 0.17681461199754267


In [41]:
# check it again
starttime = timeit.default_timer()
print("The start time is :",starttime)
data.groupBy('follower_count').count().orderBy("count", ascending=False).show()
print("The time difference is :", timeit.default_timer() - starttime)


The start time is : 17636.764051849
+--------------+-----+
|follower_count|count|
+--------------+-----+
|       2000000|  374|
|          2000|  325|
|          4000|  250|
|        942000|  244|
|          5000|  232|
|          7000|  227|
|          3000|  219|
|          1000|  219|
|       5000000|  215|
|       1000000|  178|
|          6000|  172|
|          9000|  141|
|          8000|  126|
|         28000|  115|
|         10000|  113|
|         20000|   99|
|         19000|   93|
|         13000|   90|
|        190000|   90|
|         11000|   88|
+--------------+-----+
only showing top 20 rows

The time difference is : 0.8526418090004881


In [42]:
# cast follower_count column type as int
starttime = timeit.default_timer()
print("The start time is :",starttime)
data = data.withColumn("follower_count", F.col("follower_count").cast("int"))
df.show(2)
print("The time difference is :", timeit.default_timer() - starttime)



The start time is : 17637.683214949
+----------+--------------------+----------+--------------+--------------------+-----+-----------------+-------------+--------------------+--------------------+--------------------+--------------------+
|  category|         description|downloaded|follower_count|           image_src|index|is_image_or_video|  poster_name|       save_location|            tag_list|               title|           unique_id|
+----------+--------------------+----------+--------------+--------------------+-----+-----------------+-------------+--------------------+--------------------+--------------------+--------------------+
|   tattoos|Elmira Kruger sha...|         1|           848|https://i.pinimg....| 8822|            image|Alicia Keller|Local save in /da...|Arm Sleeve Tattoo...|Elmira Kruger on ...|db80a4e4-293a-45c...|
|home-decor|Вместе с Русланом...|         1|           35k|https://i.pinimg....| 5996|            image|  INMYROOM.RU|Local save in /da...|Scandinavian 

In [43]:
# cassandra not allow use index as a key name, change "index" to "idx"
starttime = timeit.default_timer()
print("The start time is :",starttime)
data = data.withColumnRenamed('index', 'idx')

print("The time difference is :", timeit.default_timer() - starttime)


The start time is : 17638.319509226
The time difference is : 0.008557250999729149


In [44]:
# reorder selected columns
starttime = timeit.default_timer()
print("The start time is :",starttime)
data = data.select('idx', 'title', 'poster_name', 'category', 'follower_count', 'description', 'image_src', 'is_image_or_video', 'tag_list', 'unique_id')
print("The time difference is :", timeit.default_timer() - starttime)


The start time is : 17638.391884772
The time difference is : 0.03433494299679296


In [45]:
# reorder dataset with acsending index
starttime = timeit.default_timer()
print("The start time is :",starttime)
data = data.orderBy("idx", ascending=True)
print("The time difference is :", timeit.default_timer() - starttime)



The start time is : 17638.476349367
The time difference is : 0.012651852000999497


In [46]:
# schema = StructType([
#     StructField("index",IntegerType(),True), \
#     StructField("title",StringType(),True), \
#     StructField("poster_name",StringType(),True), \
#     StructField("category", StringType(), True), \
#     StructField("follower_count", IntegerType(), True), \
#     StructField("description", StringType(), True), \
#     StructField("downloaded",IntegerType(),True), \
#     StructField("image_src",StringType(),True), \
#     StructField("is_image_or_video", StringType(), True), \
#     StructField("save_location", StringType(), True), \
#     StructField("tag_list", StringType(), True),  \
#     StructField("unique_id",StringType(),True), \
#         ])
# df = spark.createDataFrame(data=df.collect(),schema=schema)

In [47]:
starttime = timeit.default_timer()
print("The start time is :",starttime)
data.write.format("org.apache.spark.sql.redis").option("table", "pins_redis").option("key.column", "idx").save()
print("The time difference is :", timeit.default_timer() - starttime)


The start time is : 17638.614986063
The time difference is : 1.464999228999659


### Send data to Cassandra

In [48]:
# reloading data in spark-redis ready to save to cassandra
starttime = timeit.default_timer()
print("The start time is :",starttime)
df_redis = spark.read.format("org.apache.spark.sql.redis").option("table", "pins_redis").option("key.column", "index").load()
print("The time difference is :", timeit.default_timer() - starttime)


In [49]:
# initialise cassandra driver
cluster = Cluster()
session = cluster.connect()

In [ ]:
# create a cassandra keyspace
session.execute("CREATE KEYSPACE pinterest_project WITH replication = {'class':'SimpleStrategy', 'replication_factor' : 3};")

In [62]:
try:
    session.execute("CREATE KEYSPACE pinterest_project WITH replication = {'class':'SimpleStrategy', 'replication_factor' : 3};")
except:
    pass


In [54]:
# initialise named cassandra key space
session.execute("USE pinterest_project;")

In [65]:
try:
    session.execute("DROP TABLE pinterest")
except:
    pass

In [67]:
session.execute("CREATE TABLE pinterest(idx int, title text, poster_name text, category text, follower_count int, description text, image_src text, is_image_or_video text, tag_list text, unique_id text PRIMARY KEY);")


In [69]:
# make preparedUpdate statements
starttime = timeit.default_timer()
print("The start time is :",starttime)
preparedUpdate = session.prepare(
    """ 
    INSERT INTO pinterest (idx, title, poster_name, category, follower_count, description, image_src, is_image_or_video, tag_list, unique_id) VALUES(?, ?, ?, ?, ?, ?, ?, ?, ?, ?);
    """
)


# write df to cassandra
for item in df_redis.collect():
    session.execute(preparedUpdate, [item[0], item[1], item[2], item[3], item[4], item[5], item[6], item[7], item[8], item[9]])
print("The time difference is :", timeit.default_timer() - starttime)




The start time is : 18436.57577475
The time difference is : 7.913507859000674


In [ ]:
# Read pinterest table in cassandra
results = session.execute("SELECT * FROM pinterest")

In [ ]:
# check if table in cassandra has read
# print first row of the table
print(results[0])

Row(idx=4317, category='education', description='Women in the Modern World : Annals of the American Academy of Political and Social Science, V143, May, 1929', follower_count=2000000, image_src='https://i.pinimg.com/originals/d9/85/5e/d9855ea66daf142e7a8e8d3671dfe1af.jpg', is_image_or_video='image', poster_name='Walmart', tag_list='Way Of Life,The Life,Inspiring Meaning,John Calvin,Battle Of Waterloo,National Convention,Move Mountains,Public Speaking,Human Nature', title='Women in the Modern World: Annals of the American Academy of Political and Social Science, V143, May, 1929 (Paperback) - Walmart.com', unique_id='ebce1b3d-149c-498d-bef6-e5538dc4d341')


/tmp/ipykernel_51283/302529547.py:2: DeprecationWarning: ResultSet indexing support will be removed in 4.0. Consider using ResultSet.one() to get a single row.
  print(results[0])


In [ ]:
# shutdown cassandra session and cluster

session.cluster.shutdown()
session.shutdown()

In [ ]:
# stop spark
spark.stop()